# 딥러닝을 적용한 추천 시스템
- 딥러닝을 어떻게 추천에 적용할 수 있는지 생각해 보면, 우선 추천을 위한 입력으로 사용자와 아이템의 특성을 나타낼 수 있는 embedding 데이터가 필요하다. 사용자와 아이템을 embidding해서 입력으로 넣으면 사용자와 아이템 간의 특성을 학습해서 추천(예측)에 이용할 수 있다.

- 이와 같이 One-hot representation 으로 변환하고 (input layer), 이를 잠재요인(Embedding Layer)으로 연결시킨다. 그러면 사용자 잠재요인(User latent Vector)과 아이템 잠재요인 (Item Latent vector)이 만들어지는데, 이둘을 합쳐서 신경망의 첫번째 층인 레이어 1이 만들어진다. 사용자 잠재요인과 아이템 잠죠요인을 합치는 가장 단순한 방법은 인공신경망에서 사용되는 결합(concatenate)을 이용해서 앞뒤로 그래도 붙이는 방법이다. 일단 첫 번째 층이 만들어지면, 그 다음부터는 일반적인 DL에서와 같이 적절한 층을 쌓아나가면 된다. 맨 끝(위)에는 출력으로 예측하고자 하는 평점이 놓이기 된다. 

In [ ]:
import pandas as pd
import numpy as np

from sklearn.utils import shuffle

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dot, Add, Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD, Adam, Adamax
from tensorflow.keras.layers import Dense, Concatenate, Activation

In [ ]:
# csv 파일에서 불러오기
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('data/u.data', names=r_cols,  sep='\t',encoding='latin-1')
ratings = ratings[['user_id', 'movie_id', 'rating']].astype(int)

# train test 분리
TRAIN_SIZE = 0.75
ratings = shuffle(ratings)
cutoff = int(TRAIN_SIZE * len(ratings))
ratings_train = ratings.iloc[:cutoff]
ratings_test = ratings.iloc[cutoff:]

u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('data/u.data', sep='|', names=u_cols, encoding='latin-1')
users = users[['user_id', 'occupation']]

In [ ]:
# Convert occupation(string to integer)
occupation = {}
def convert_occ(x):
    if x in occupation:
        return occupation[x]
    else:
        occupation[x] = len(occupation)
        return occupation[x]
users['occupation'] = users['occupation'].apply(convert_occ)

In [ ]:
L = len(occupation)
train_occ = pd.merge(ratings_train, users, on='user_id')['occupation']
test_occ = pd.merge(ratings_test, users, on='user_id')['occupation']

In [ ]:
# Variable 초기화 
K = 200                             # Latent factor 수 (잠재요인 수)
mu = ratings_train.rating.mean()    # 전체 평균 
M = ratings.user_id.max() + 1       # Number of users
N = ratings.movie_id.max() + 1      # Number of movies

# Defining RMSE measure
def RMSE(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

In [ ]:
# Keras model
user = Input(shape=(1, ))                                               # User input
item = Input(shape=(1, ))                                               # Item input
P_embedding = Embedding(M, K, embeddings_regularizer=l2())(user)        # (M, 1, K)
Q_embedding = Embedding(N, K, embeddings_regularizer=l2())(item)        # (N, 1, K)
user_bias = Embedding(M, 1, embeddings_regularizer=l2())(user)          # User bias term (M, 1, )
item_bias = Embedding(N, 1, embeddings_regularizer=l2())(item)          # Item bias term (N, 1, )

In [ ]:
# Concatenate layers
P_embedding = Flatten()(P_embedding)
Q_embedding = Flatten()(Q_embedding)
user_bias = Flatten()(user_bias)
item_bias = Flatten()(item_bias)

occ = Input(shape=(1, ))
occ_embedding = Embedding(L, 3, embeddings_regularizer=l2())(occ)
occ_layer = Flatten()(occ_embedding)
R = Concatenate()([P_embedding, Q_embedding, user_bias, item_bias, occ_layer])

In [ ]:
# Neural network
R = Dense(2048)(R) # 노드가 2048개인 dense layer 추가
R = Activation('linear')(R) 
R = Dense(256)(R)  # 노드가 256개인 dense layer 추가
R = Activation('linear')(R)
R = Dense(1)(R)

In [ ]:
model = Model(inputs=[user, item, occ], outputs=R)
model.compile(
  loss=RMSE,
  optimizer=SGD(),
  #optimizer=Adamax(),
  metrics=[RMSE]
)
model.summary()

In [ ]:
# Model fitting
result = model.fit(
  x=[ratings_train.user_id.values, ratings_train.movie_id.values],
  y=ratings_train.rating.values - mu,
  epochs=65,
  batch_size=512,
  validation_data=(
    [ratings_test.user_id.values, ratings_test.movie_id.values],
    ratings_test.rating.values - mu
  )
)

In [ ]:
# Plot RMSE
import matplotlib.pyplot as plt
plt.plot(result.history['RMSE'], label="Train RMSE")
plt.plot(result.history['val_RMSE'], label="Test RMSE")
plt.xlabel('epoch')
plt.ylabel('RMSE')
plt.legend()
plt.show()

In [ ]:
# Prediction
user_ids = ratings_test.user_id.values[0:6]
movie_ids = ratings_test.movie_id.values[0:6]
predictions = model.predict([user_ids, movie_ids]) + mu
print("Actuals: \n", ratings_test[0:6])
print()
print("Predictions: \n", predictions)

In [ ]:
# 정확도(RMSE)를 계산하는 함수 
def RMSE2(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

user_ids = ratings_test.user_id.values
movie_ids = ratings_test.movie_id.values
y_pred = model.predict([user_ids, movie_ids]) + mu
y_pred = np.ravel(y_pred, order='C')
y_true = np.array(ratings_test.rating)

RMSE2(y_true, y_pred)